In [2]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [3]:
AGENT_NAME = "s010-multiturn"

In [4]:
# # (Optional) Create a new task. If you don't create a new task, each message will be sent to a new task. The server will create the task for you.

# import uuid

# TASK_ID = str(uuid.uuid4())[:8]

# rpc_response = client.agents.rpc_by_name(
#     agent_name=AGENT_NAME,
#     method="task/create",
#     params={
#         "name": f"{TASK_ID}-task",
#         "params": {}
#     }
# )

# task = rpc_response.result
# print(task)

In [5]:
# Test non streaming response
from typing import List, cast
from agentex.types import TaskMessage, TextContent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="message/send",
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "stream": False
    }
)

# # Extract and print just the text content from the response
# # The response is expected to be a dict with a "result" key containing a list of message dicts
if rpc_response and rpc_response.result:

    # We know that the result of the message/send when stream is set to False will be a list of TaskMessage objects
    task_message_list = cast(List[TaskMessage], rpc_response.result)
    for task_message in rpc_response.result:
        if isinstance(task_message, TaskMessage):
            content = task_message.content
            if isinstance(content, TextContent):
                text = content.content
                print(text)
        else:
            print(f"Found non-text {type(task_message)} object in response.")


Hello! I can assist you with a variety of tasks, including:

1. Answering questions on a wide range of topics, including science, history, technology, and more.
2. Providing explanations or summaries of complex concepts.
3. Offering writing assistance, such as proofreading, editing, or generating ideas for essays and articles.
4. Helping with problem-solving in areas like math, coding, or logic puzzles.
5. Engaging in conversation to provide companionship or entertainment.

If there's something specific you need help with, feel free to ask!


In [6]:
# Test streaming response
import json
from agentex.types import AgentRpcResponse
from agentex.types.agent_rpc_result import StreamTaskMessageDelta, StreamTaskMessageFull
from agentex.types.text_delta import TextDelta
from agentex.types.task_message_update import TaskMessageUpdate


# The result object of message/send will be a TaskMessageUpdate which is a union of the following types:
# - StreamTaskMessageStart: 
#   - An indicator that a streaming message was started, doesn't contain any useful content
# - StreamTaskMessageDelta: 
#   - A delta of a streaming message, contains the text delta to aggregate
# - StreamTaskMessageDone: 
#   - An indicator that a streaming message was done, doesn't contain any useful content
# - StreamTaskMessageFull: 
#   - A non-streaming message, there is nothing to aggregate, since this contains the full message, not deltas

# Whenn processing StreamTaskMessageDelta, if you are expecting more than TextDeltas, such as DataDelta, ToolRequestDelta, or ToolResponseDelta, you can process them as well
# Whenn processing StreamTaskMessageFull, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

with client.agents.with_streaming_response.rpc_by_name(
    agent_name=AGENT_NAME,
    method="message/send",
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "stream": True
    }
) as response:
    for agent_rpc_response_str in response.iter_text():
        chunk_rpc_response = AgentRpcResponse.model_validate(json.loads(agent_rpc_response_str))
        # We know that the result of the message/send when stream is set to True will be a TaskMessageUpdate
        task_message_update = cast(TaskMessageUpdate, chunk_rpc_response.result)

        # Print oly the text deltas as they arrive or any full messages
        if isinstance(task_message_update, StreamTaskMessageDelta):
            delta = task_message_update.delta
            if isinstance(delta, TextDelta):
                print(delta.text_delta, end="", flush=True)
            else:
                print(f"Found non-text {type(task_message)} object in streaming message.")
        elif isinstance(task_message_update, StreamTaskMessageFull):
            content = task_message_update.content
            if isinstance(content, TextContent):
                print(content.content)
            else:
                print(f"Found non-text {type(task_message)} object in full message.")


Hello! I can assist you with a wide range of tasks, including:

1. **Answering questions**: I can provide information on a variety of topics, including history, science, technology, and more.
2. **Learning and education**: I can help explain concepts, provide summaries, and assist with studying.
3. **Writing assistance**: I can help you draft, edit, or brainstorm ideas for essays, reports, and creative writing.
4. **Programming help**: I can assist with coding questions, debugging, and providing explanations of programming concepts.
5. **Language translation**: I can translate text between several languages.
6. **General advice**: I can offer suggestions on topics like time management, study techniques, and more.

Feel free to ask me anything specific you need help with!
